# myhmmlearn
刘建平Blog: https://www.cnblogs.com/pinard/p/6955871.html

In [51]:
# -*- coding: utf-8 -*-
# Created on Mon Sep 30 09:59:48 2019

import numpy as np

class MultinomialHMM:
    def __init__(self):
        self.n_states = None
        self.n_iter = None
        self.tol = None
    
        self.transmits = np.array([[0.5, 0.2, 0.3],
                               [0.3, 0.5, 0.2],
                               [0.2, 0.3, 0.5]])      # transmits[i][j]: 从状态i转移到j的概率
        
        self.emits = np.array([ [0.5, 0.4, 0.7],
                                [0.5, 0.6, 0.3] ])            # emits[i][j]: 观测i被隐状态j发射的概率
        
        self.start_prob = np.array([0.2, 0.4, 0.4]) 
    
    def fit(self, observs):     # 根据观测值用鲍姆韦尔奇算法学习模型参数, 转移发射起始
        pass
    
    def decode(self, observations):         # 根据观测得到最大概率隐状态序列及其分数
        all_route = []
        forward_score = self.start_prob * self.emits[observations[0]]
        for t in range(1, len(observations)):
            trans_score = forward_score * self.transmits.T
            all_route.append(np.argmax(trans_score, 1))
            forward_score = np.max(trans_score, 1) * self.emits[observations[t]]
        
        all_route.reverse()
        next_route = np.argmax(forward_score)     # 最后1个隐状态
        final_route = [next_route]                # 回溯路径
        for route in all_route:
            next_route = route[next_route]
            final_route.append(next_route)
        
        final_route.reverse()
        max_route_score = np.log(np.max(forward_score))
        return final_route, max_route_score
    
    def score(self, observations):          # 模型学习完成, 对观察序列作出评分
        forward_score = self.start_prob * self.emits[observations[0]]
        for i in range(1, len(observations)):
            forward_score = np.matmul(forward_score, self.transmits)   # 转移后的前向分数
            forward_score *= self.emits[observations[i]]    # 转移后各状态发射出当前观测的前向分数
        
        return np.log(np.sum(forward_score))    # 返回的分数是loge(score)

In [54]:
hmm = MultinomialHMM()
seen = np.array([0, 1, 1])
score = hmm.score(seen)
score      # 正确, 与刘建平博客实例结果一致
hidden_states, score = hmm.decode(seen)
hidden_states

[2, 1, 1]